# email-as-browser

Version 2.1

This project allows users to use an email as a browser. <br> Using email-as-browser will greatly [1] improve the security and [2] reduce the setup cost.

In [1]:
# Extra Packages and Function
#
import pdfkit
from os.path import basename
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate

from email.header import decode_header

def checkSubjectUTF8(text):
    if len(text)>=7 and text[0:6]=='=?big5':
        print("big5 encode")
        text=decode_header(message["Subject"])[0][0].decode('big5')
    if len(text)>=9 and text[0:8]=='=?gb2312':
        print("gb2312 encode")
        text=decode_header(message["Subject"])[0][0].decode('gb2312')
    if len(text)>=14 and text[0:13]=='=?iso-2022-jp':
        print("iso-2022-jp encode")
        text=decode_header(message["Subject"])[0][0].decode('iso-2022-jp')     
    return(text) 

def checkPatternString(text, pattern='http://'):
    pat="(^| )"+pattern+"(.*)"
    commandLen=len(pattern)
    x = re.search(pat, text)
    if x:
        return(1)
    return(0)

def extractPatternString(text, pattern='http://'):
    pat="(^| )"+pattern+"(.*)"
    commandLen=len(pattern)
    x = re.search(pat, text)
    if x:
        extractText=x.group(0)
        para=extractText[commandLen:].strip()
        return(para)

In [27]:
import requests
import re
import bs4
import pandas as pd

import os
import sys
import time
import imaplib
import email
from datetime import datetime
import re
import base64
import traceback 
from datetime import datetime as dt


import smtplib
import sys

from bs4 import BeautifulSoup 

conf_file='/media/zorin/casper-rw/prj/email-as-browser/email_as_browser_conf.xml'
ver="Version: 2023-04-1"
debug=0
print(ver+"  debug :"+str(debug+0))

Version: 2023-04-1  debug :0


In [3]:
# Reading the data inside the xml file to a variable under the name  data
with open(conf_file , 'r') as f:
    data_conf = f.read() 
bs_data = BeautifulSoup(data_conf, 'xml')

smtp_server  = bs_data.find('data', {'name':'smtp_server'}).text
smtp_port    = bs_data.find('data', {'name':'smtp_port'}).text
smtp_login   = bs_data.find('data', {'name':'smtp_login'}).text
smtp_pwd     = bs_data.find('data', {'name':'smtp_pwd'}).text

TO_MAIL      = bs_data.find('data', {'name':'TO_MAIL'}).text
FM_MAIL      = bs_data.find('data', {'name':'FM_MAIL'}).text

IMAP_username= bs_data.find('data', {'name':'IMAP_username'}).text
IMAP_password= bs_data.find('data', {'name':'IMAP_password'}).text
IMAP_SERVER  = bs_data.find('data', {'name':'IMAP_SERVER'}).text
IMAP_PORT    = bs_data.find('data', {'name':'IMAP_PORT'}).text


smtp_port=int(smtp_port)
IMAP_PORT=int(IMAP_PORT)

In [4]:
if 1==2:
    print(smtp_server)
    print(smtp_port)    
    print(smtp_login)
    print(smtp_pwd)    
    print(TO_MAIL)
    print(FM_MAIL)    
    print(IMAP_username)
    print(IMAP_password) 
    print(IMAP_SERVER) 
    print(IMAP_PORT) 

In [6]:
# REF: https://stackoverflow.com/questions/3362600/how-to-send-email-attachments
def mymail(to, subject, body, fm, files=None):
    #body = '<font face="Consolas" size="10">' + body + '</font>'
    msg = email.message_from_string(body)
    msg = MIMEMultipart()
    msg['From'] = fm
    msg['To'] = to
    msg['Date'] = formatdate(localtime=True) #new
    msg['Subject'] = subject
    msg.attach(MIMEText(body))
    
    for f in files or []:
        print("attachment for "+f)
        with open(f, "rb") as fil:
            part = MIMEApplication(
                fil.read(),
                Name=basename(f)
            )
        # After the file is closed
        part['Content-Disposition'] = 'attachment; filename="%s"' % basename(f)
        msg.attach(part)
        
    
    s = smtplib.SMTP(smtp_server,smtp_port)
    #s = smtplib.SMTP("smtp.gmail.com",587)
    #s = smtplib.SMTP("smtp-mail.outlook.com",587)

    s.ehlo() # Hostname to send for this command defaults to the fully qualified domain name of the local host.
    s.starttls() #Puts connection to SMTP server in TLS mode
    s.ehlo()
    s.login(smtp_login, smtp_pwd)  #app passsword
    s.sendmail(fm, to,msg.as_string())

 

In [8]:

###################################################################### CUSTOMIZATION START
outputfile='/media/zorin/casper-rw/prj/email-as-browser/out.pdf'

def cmdQuery(query, qType==2):
    if qType!=2:
       query=query.replace(' ', '+')
       targetURL="https://www.google.com/search?client=firefox-b-d&q="+query
       imageTab="&hl=xx-elmer&tbm=isch&sxsrf=APwXEdcm2r14Jv4KLoScGrD-QI3nCbPL7Q%3A1680750182628&source=hp&biw=1600&bih=757&ei=ZjYuZLO5JOjP2roPi5eFiA4&iflsig=AOEireoAAAAAZC5Edg3Tx0HwXX4YxaseUjrfyAjAKzfs&oq=shenzhen+map&gs_lcp=CgNpbWcQARgAMgcIABCABBATMgcIABCABBATMgcIABCABBATMgcIABCABBATMggIABAIEB4QEzIICAAQCBAeEBMyCAgAEAgQHhATMggIABAIEB4QEzIICAAQCBAeEBMyCAgAEAgQHhATUABYAGDnD2gAcAB4AIABQIgBQJIBATGYAQCqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img"
       if qType==1:
            targetURL=targetURL+imageTab
            
    print(targetURL)
    try:
        pdfkit.from_url(targetURL, outputfile)
    except OSError as e:
        if 'Done' not in str(e):
            #raise e
            print("Ignore OSError")
    now = str(datetime.now())[1:19]        
    mymail(TO_MAIL, query, now+"   URL: "+targetURL , FM_MAIL, [outputfile] )



###################################################################### CUSTOMIZATION END

In [ ]:
#mymail('kimman.xxxxx@xxxxx.com','test', 'body',FM_MAIL) 



In [9]:
def waitMin(min):
    ###60 sec=1 min
    #sec=minx60
    time.sleep(60)

def handleJunkBox():
    statusj, countj = M.select('junk')
    _, message_numbers_raw = M.search(None, 'ALL')
    junkMsg=message_numbers_raw[0].split()
    if len(junkMsg)>0:
        print("Moving Junk Msg "+str(len(junkMsg)))
        for i in range(len(junkMsg)):
            msg_uid=junkMsg[0]
            M.store(msg_uid, '-FLAGS', '\\SEEN') 
            result = M.copy( msg_uid, 'Inbox')
            #print(result[0])
            if result[0] == 'OK':
                mov, data = M.store( msg_uid , '+FLAGS', '(\\Deleted)')
                M.expunge()

In [10]:
M=imaplib.IMAP4_SSL(host=IMAP_SERVER, port=IMAP_PORT)

In [11]:
M.login(IMAP_username,IMAP_password)

('OK', [b'LOGIN completed.'])

In [12]:
handleJunkBox()

In [13]:
# get Unseen Message
status, count   = M.select('inbox')
_, message_numbers_raw = M.search(None, '(UNSEEN)')
unseenMsg=message_numbers_raw[0].split()
print(unseenMsg)

[]


In [15]:
fromWho='X'
subject='X'
dieFlag=0
firstFlag=0
showNumFreqWord=100
subjectEmailLast=''

#for loopdx in range(5):
subjectLast="---"
fromWhoLast="---"
datetimeLast=dt.now()
while 1==1:
    
    #The program will stop at 59 as a scheduler will start the program each hour again. 
    stamp=dt.now().minute 
    if stamp >= 59 and firstFlag==1:   
        sys.exit()
    if dieFlag==1:
        sys.exit()
    if debug==1:
        print("timestamp:"+str(stamp))
    firstFlag=1
    ERRORTAG=''

    try:  
        ERRORTAG=' IMAP'
        for messageNumber in unseenMsg:
            
            print("an email...arrived... ")
            
            _, msg = M.fetch(messageNumber, '(RFC822)')    
            message = email.message_from_bytes(msg[0][1])
            M.store(messageNumber, '+FLAGS', '\SEEN') 
            fromWho=str(message["from"])
            toWho=str(message["to"])
            subject=str(message["subject"])
            status, data = M.fetch(messageNumber,'(UID BODY[TEXT])')
            raw_email=data[0][1]
            datetimeNow=dt.now()
            diff=datetimeNow-datetimeLast
            diff_min=(diff.days * 24 * 60) + (diff.seconds/60)
            goFlag=1
            if fromWhoLast==fromWho and subjectLast==subject and diff_min<1 :
                print("consider this email as duplicated")
                goFlag=0
            fromWhoLast=fromWho
            subjectLast=subject
            datetimeLast=datetimeNow
            
            subject = subject.strip()
            subject=checkSubjectUTF8(subject)
            subjectorg = subject
                    
###################################################################### CUSTOMIZATION START
            if goFlag==1 :
                if "noreply" not in fromWho:               ### Elimiate POTENTIAL SPAM 


                    #subject = subject.lower()
                    print("subject: "+subject)
                    
                    intendedAction="Ignore this email request as expired (over mins) :"
                    ERRORTAG=' Email-As-Browser'
                    timeflag=0
                    pat="time(\d\d.\d\d.\d\d.\d\d.\d\d)"
                    commandLen=len('time')
                    x = re.search(pat, subject)
                    if x:
                        extractText=x.group(0)
                        para=extractText[commandLen:].strip()
                        print(para)
                        stamp=datetime.strptime( para, "%y.%m.%d.%H.%M")             
                        now=datetime.now()
                        c=now - stamp
                        diffmins = divmod(c.total_seconds(), 60) 
                        timeflag=1
                        if diffmins[0]>15:
                            print(intendedAction+str(diffmins[0]) )
                            mymail(TO_MAIL, "timeout : "+subject, "timeout "+str(diffmins[0]) , FM_MAIL)
                            continue 
                    else:
                        print("No time parameter")


                    intendedAction="Additional Email Address :";
                    emailaddr="XXX"
                    match = re.search(r'[\w.+-]+@[\w-]+\.[\w.-]+', subject)
                    if match:
                        emailaddr=match.group(0)
                        print(emailaddr)
                    else:
                        print(intendedAction+" No")


                    pat="(^| )cmd(.*)"
                    commandLen=len('cmd')
                    x = re.search(pat, subject.lower())
                    if x:
                        extractText=x.group(0)
                        extractText=extractText.strip()
                        para=extractText[commandLen:].strip()
                        ELEL = para.split(',')
                        for element in ELEL:
                            print("cmd:"+element)
                            if checkPatternString(element, 'http://' )==1:
                                qType=2
                                cmdQuery(element, qType)
                            elif checkPatternString(element, 'https://' )==1:
                                qType=2
                                cmdQuery(element, qType)
                            else:
                                qType=0
                                cmdQuery(element, qType)
                            time.sleep(1)
                            os.remove(outputfile)
                            dieFlag=1
                    else:
                        print("No cmd tag found")

                        
                    
                    pat="(^| )image(.*)"
                    commandLen=len('image')
                    x = re.search(pat, subject.lower())
                    if x:
                        extractText=x.group(0)
                        extractText=extractText.strip()
                        para=extractText[commandLen:].strip()
                        ELEL = para.split(',')
                        for element in ELEL:
                            print("image:"+element) 
                            qType=1
                            cmdQuery(element, qType)
                            time.sleep(1)
                            os.remove(outputfile)
                            dieFlag=1
                    else:
                        print("No image tag found")

 ###################################################################### CUSTOMIZATION END
            
        now = datetime.now()
        current_time = now.strftime("%H:%M")    
        print("wait ("+current_time+")")    
        if debug==1:
            break 
        waitMin(1)  
        fromWho='X'
        subject='X'
        #handleJunkBox
        handleJunkBox()
        #get Unseen Message
        status, count   = M.select('inbox')
        _, message_numbers_raw = M.search(None, '(UNSEEN)')
        unseenMsg=message_numbers_raw[0].split()  
    except:
        #traceback.print_exc() 
        exceptMsg= traceback.format_exc()
        print(exceptMsg)
        print("Exception Occurred")
        if ERRORTAG!=' IMAP':
            subjectEmail="Exception Occurred :"+ERRORTAG+"("+subject+")"
            if subjectEmail!=subjectEmailLast:
                mymail(TO_MAIL, subjectEmail, "Exception Occurred:"+exceptMsg , FM_MAIL)
                subjectEmailLast=subjectEmail
            if fromWho!='X':
                print("problem with this email:"+fromWho+", "+subject) 
        waitMin(1)
        M=imaplib.IMAP4_SSL(host=IMAP_SERVER)
        M.login(IMAP_username,IMAP_password)
        status, count = M.select('inbox')
        print("Exception End")

    if debug==1: 
        break

print("-END-")

timestamp:20
wait (23:20)
timestamp:21
an email...arrived... 
subject: cmd https://www.ctfeshop.com.hk/special/gp.aspx
No time para
Additional Email Address : No
cmd:https://www.ctfeshop.com.hk/special/gp.aspx
https://www.ctfeshop.com.hk/special/gp.aspx
attachment for /media/zorin/casper-rw/prj/email-as-browser/out.pdf
No image cmd
wait (23:22)


SystemExit: 

/home/zorin/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
